<a href="https://colab.research.google.com/github/jacksonbrandberg/patent-landscaping/blob/main/patent-and-research-landscaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up with BigQuery
[This notebook](https://colab.research.google.com/notebooks/bigquery.ipynb#scrollTo=ONI1Xo0-KtAD) provides a great tutorial on how to set up BigQuery access with Google Colab

In [ ]:
import pandas as pd
import re
import bs4
from bs4 import BeautifulSoup

In [ ]:
#mount drive
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
#enter project id
project_id = 'patent-classification-313315'

In [ ]:
#import bigquery
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# Two options:
The user can either upload a csv file to their drive that contains the publication numbers of active patents they want to analyze. The file should be formatted with "publication_number" as the column name, and the patents should be formatted with "US-patent_number-patent_kind_code", ex: "US-123456-A1".
Alternatively, the user can query patents based on a keyword phrase they provide. It will then return all patent matches that contain that phrase.

## Option 1

In [ ]:
# load in dataframe of pub numbers of desired patents to work with 
# we are working with patents in pharmacology and patient insurance
# mount to drive and find file path for appropriate project
df = pd.read_csv("/content/drive/MyDrive/Practicum/sample_pub_numbers.csv")
df.head()

In [ ]:
dest_dataset = 'desired_publication_numbers'
# Upload these to our dataset on BigQuery.
ID_table = 'ID_table'
full_table_path = '{}.{}'.format(dest_dataset, ID_table)
df.to_gbq(full_table_path, project_id, if_exists = "replace")

In [ ]:
df = client.query('''
#standardSQL

WITH P AS (
  SELECT 
  publication_number, 
  floor(priority_date / 10000) priority_yr
  FROM `patents-public-data.patents.publications`
  WHERE country_code = 'US'
  AND floor(priority_date / 10000) >= 1950
  AND country_code = 'US'
)

SELECT 
P.publication_number,
P.priority_yr,
abstracts.text
FROM `patents-public-data.patents.publications` as pubs,
UNNEST(abstract_localized) as abstracts
JOIN P 
  ON P.publication_number = pubs.publication_number
JOIN `patent-classification-313315.desired_publication_numbers.ID_table` my_pubs
  ON pubs.publication_number = my_pubs.publication_number
WHERE abstracts.language = 'en'
''').to_dataframe()

## Option 2

In [ ]:
df = client.query('''
  SELECT * FROM
  `patents-public-data.patents.publications` AS patentsdb,
  UNNEST(abstract_localized) AS abstract_info
  # enter query phrase to search patents for
  # in this example, we use the word "virus"
  WHERE LOWER(abstract_info.text) LIKE '%virus%'
    AND patentsdb.country_code = 'US'
  ORDER BY patentsdb.priority_date DESC
  # enter max number of patents to analyze in results
  LIMIT 200
''').to_dataframe()

In [ ]:
df.head()

In [ ]:
abstracts = df["abstract_localized"]
abstracts_text = []
for i in abstracts:
  for j in i:
    abstract = j["text"]
    abstract = abstract.replace("\n", "")
    abstract = abstract.replace("&#39;", "'")
    abstract = abstract.replace("/", " or ")
    abstract = re.sub("[\(\[].*?[\)\]]", "", abstract)
    if abstract not in abstracts_text:
      abstracts_text.append(abstract)
    else:
      pass

In [ ]:
abstracts_text[0:3]

## Optional
Here you can add your own idea for a patent abstract and see where it falls within the landscape

In [ ]:
#enter text below
your_patent = "Enter your patent here"
abstracts_text.append(your_patent)

# Import BERT Model and Encode Messages
Hugging face has [several pretrained BERT models](https://huggingface.co/models). You can use one trained on patents for general use. In our case, one on biomedical data might be more applicable. It is easy to import the different models, so find one that seems to work best for you

In [ ]:
import tensorflow as tf

In [ ]:
! pip install -Uq sentence-transformers

In [ ]:
import pandas as pd
import scipy
import numpy as np
import sklearn
from sentence_transformers import models, SentenceTransformer, util
from sklearn.cluster import KMeans
from sentence_transformers import util
from sklearn.metrics.pairwise import cosine_similarity
# choose a model relevant to this project
# "AI-Growth/PatentSBERTa is a great general patent BERT model"
# for this task, we're using a model trained on biomedical abstracts, which is relevant to our research
model = SentenceTransformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')

#Create Clusters

In [ ]:
# change number below to adjust amount of clusters
clusters = 5
sentence_embeddings = model.encode(abstracts_text)
kmeans = KMeans(n_clusters=clusters).fit(sentence_embeddings)
kmeans.labels_

Add labels. Run the chunk below based on whether or not you included an original patent

In [ ]:
combined_dataset = []
# use this if using an original patent
for i in range(len(abstracts_text)):
  if i == len(abstracts_text)-1:
    label = kmeans.labels_[i]
    abstract = abstracts_text[i]
    source = "patent original"
    instance = (abstract, source, label)
    combined_dataset.append(instance)
  else:
    label = kmeans.labels_[i]
    abstract = abstracts_text[i]
    source = "patent"
    instance = (abstract, source, label)
    combined_dataset.append(instance)

In [ ]:
combined_dataset = []
# use this if not using an original patent
for i in range(len(abstracts_text)):
  label = kmeans.labels_[i]
  abstract = abstracts_text[i]
  source = "patent"
  instance = (abstract, source, label)
  combined_dataset.append(instance)

# Import New Research and Apply to Existing Clusters
Google has many [public datasets](https://cloud.google.com/bigquery/public-data). Feel free to find one relevant to the patents you are landscaping! In this demo, we are using biomedical research on covid-19

In [ ]:
#can replace arxiv with other research database
research_df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.breathe.arxiv`
''').to_dataframe()

In [ ]:
research_abstracts = research_df["abstract"]
research_abstracts_clean = []
#remove \n that is in the text
for i in research_abstracts:
  abstract = i.replace("\n", "")
  abstract = abstract.replace("&#39;", "'")
  abstract = abstract.replace("/", " or ")
  abstract = re.sub("[\(\[].*?[\)\]]", "", abstract)
  research_abstracts_clean.append(abstract)

In [ ]:
#research_sample = research_abstracts_clean[1:10]
research_embeddings = model.encode(research_abstracts_clean)
research_labels = kmeans.predict(research_embeddings)

In [ ]:
for i in range(len(research_abstracts_clean)):
  label = research_labels[i]
  abstract = research_abstracts_clean[i]
  source = "research"
  instance = (abstract, source, label)
  combined_dataset.append(instance)

# Create combined dataset

In [ ]:
combined_df = pd.DataFrame(combined_dataset, columns =['Abstract', 'Source', 'Label'])
combined_df.head()

In [ ]:
combined_df["Source"].value_counts()

#Visualize Results

In [ ]:
import matplotlib
from matplotlib import pylab as plt
pd.value_counts(combined_df['Label']).plot.bar()
plt.ylabel('Count')
plt.xlabel('Cluster Label')
plt.show()

# Download Created Dataframe

In [ ]:
from google.colab import files

combined_df.to_csv('patent_and_research_abstracts.csv')
files.download('patent_and_research_abstracts.csv')